This is the last assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

Just execute all cells one after the other and you are done - just note that in the last one you should update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"

The purpose of this assignment is to learn how feature engineering boosts model performance. You will apply Discrete Fourier Transformation on the accelerometer sensor time series and therefore transforming the dataset from the time to the frequency domain. 

After that, you’ll use a classification algorithm of your choice to create a model and submit the new predictions to the grader. Done.

Please make sure you run this notebook from an Apache Spark 2.3 notebook.

So the first thing we need to ensure is that we are on the latest version of SystemML, which is 1.3.0 (as of 20th March'19) Please use the code block below to check if you are already on 1.3.0 or higher. 1.3 contains a necessary fix, that's we are running against the SNAPSHOT




If you are blow version 1.3.0, or you got the error message "No module named 'systemml'"  please execute the next two code blocks and then

# PLEASE RESTART THE KERNEL !!!

Otherwise your changes won't take effect, just double-check every time you run this notebook if you are on SystemML 1.3


In [ ]:
!pip install https://github.com/IBM/coursera/blob/master/systemml-1.3.0-SNAPSHOT-python.tar.gz?raw=true



Now we need to create two sym links that the newest version is picket up - this is a workaround and will be removed as soon as SystemML 1.3 will be pre-installed on Watson Studio once officially released.


In [ ]:
!ln -s -f ~/user-libs/python3.6/systemml/systemml-java/systemml-1.3.0-SNAPSHOT-extra.jar ~/user-libs/spark2/systemml-1.3.0-SNAPSHOT-extra.jar
!ln -s -f ~/user-libs/python3.6/systemml/systemml-java/systemml-1.3.0-SNAPSHOT.jar ~/user-libs/spark2/systemml-1.3.0-SNAPSHOT.jar

# Please now restart the kernel and start from the beginning to make sure you've installed SystemML 1.3



In [1]:
from pyspark.sql import SparkSession
# initialise sparkContext
spark = SparkSession.builder \
    .master('local') \
    .appName('myAppName') \
    .config("spark.cores.max", "2") \
    .getOrCreate()

#     .config('spark.executor.memory', '1gb') \

sc = spark.sparkContext

# using SQLContext to read parquet file
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
import pyspark.sql.functions as F

In [2]:
# !pip install systemml

In [3]:
from systemml import MLContext
ml = MLContext(spark)
print(ml.version())
    
if not ml.version() == '1.3.0-SNAPSHOT':
    print(ml.version())
#     raise ValueError('please upgrade to SystemML 1.3.0, or restart your Kernel (Kernel->Restart & Clear Output)')

1.3.0-SNAPSHOT


Let's download the test data since it's so small we don't use COS (IBM Cloud Object Store) here

In [4]:
# !wget https://github.com/IBM/coursera/blob/master/coursera_ml/shake.parquet?raw=true
# !mv shake.parquet?raw=true shake.parquet

Now it’s time to read the sensor data and create a temporary query table.

In [5]:
df=spark.read.parquet('shake.parquet')

In [6]:
df.show()

+-----+---------+-----+-----+-----+
|CLASS| SENSORID|    X|    Y|    Z|
+-----+---------+-----+-----+-----+
|    2| qqqqqqqq| 0.12| 0.12| 0.12|
|    2|aUniqueID| 0.03| 0.03| 0.03|
|    2| qqqqqqqq|-3.84|-3.84|-3.84|
|    2| 12345678| -0.1| -0.1| -0.1|
|    2| 12345678|-0.15|-0.15|-0.15|
|    2| 12345678| 0.47| 0.47| 0.47|
|    2| 12345678|-0.06|-0.06|-0.06|
|    2| 12345678|-0.09|-0.09|-0.09|
|    2| 12345678| 0.21| 0.21| 0.21|
|    2| 12345678|-0.08|-0.08|-0.08|
|    2| 12345678| 0.44| 0.44| 0.44|
|    2|    gholi| 0.76| 0.76| 0.76|
|    2|    gholi| 1.62| 1.62| 1.62|
|    2|    gholi| 5.81| 5.81| 5.81|
|    2| bcbcbcbc| 0.58| 0.58| 0.58|
|    2| bcbcbcbc|-8.24|-8.24|-8.24|
|    2| bcbcbcbc|-0.45|-0.45|-0.45|
|    2| bcbcbcbc| 1.03| 1.03| 1.03|
|    2|aUniqueID|-0.05|-0.05|-0.05|
|    2| qqqqqqqq|-0.44|-0.44|-0.44|
+-----+---------+-----+-----+-----+
only showing top 20 rows



In [7]:
type(df)

pyspark.sql.dataframe.DataFrame

In [8]:
# !pip install pixiedust

In [9]:
# import pixiedust
# display(df)

In [10]:
df.createOrReplaceTempView("df")

We’ll use Apache SystemML to implement Discrete Fourier Transformation. This way all computation continues to happen on the Apache Spark cluster for advanced scalability and performance.

In [11]:
from systemml import MLContext, dml
ml = MLContext(spark)

As you’ve learned from the lecture, implementing Discrete Fourier Transformation in a linear algebra programming language is simple. Apache SystemML DML is such a language and as you can see the implementation is straightforward and doesn’t differ too much from the mathematical definition (Just note that the sum operator has been swapped with a vector dot product using the %*% syntax borrowed from R
):

<img style="float: left;" src="https://wikimedia.org/api/rest_v1/media/math/render/svg/1af0a78dc50bbf118ab6bd4c4dcc3c4ff8502223">



In [12]:
dml_script = '''
PI = 3.141592654
N = nrow(signal)

n = seq(0, N-1, 1)
k = seq(0, N-1, 1)

M = (n %*% t(k))*(2*PI/N)

Xa = cos(M) %*% signal
Xb = sin(M) %*% signal

DFT = cbind(Xa, Xb)
'''

Now it’s time to create a function which takes a single row Apache Spark data frame as argument (the one containing the accelerometer measurement time series for one axis) and returns the Fourier transformation of it. In addition, we are adding an index column for later joining all axis together and renaming the columns to appropriate names. The result of this function is an Apache Spark DataFrame containing the Fourier Transformation of its input in two columns. 


In [13]:
from pyspark.sql.functions import monotonically_increasing_id

def dft_systemml(signal,name):
    prog = dml(dml_script).input('signal', signal).output('DFT')
    
    return (

    #execute the script inside the SystemML engine running on top of Apache Spark
    ml.execute(prog) 
     
         #read result from SystemML execution back as SystemML Matrix
        .get('DFT') 
     
         #convert SystemML Matrix to ApacheSpark DataFrame 
        .toDF() 
     
         #rename default column names
        .selectExpr('C1 as %sa' % (name), 'C2 as %sb' % (name)) 
     
         #add unique ID per row for later joining
        .withColumn("id", monotonically_increasing_id())
    )
        




Now it’s time to create individual DataFrames containing only a subset of the data. We filter simultaneously for accelerometer each sensor axis and one for each class. This means you’ll get 6 DataFrames. Please implement this using the relational API of DataFrames or SparkSQL. Please use class 1 and 2 and not 0 and 1.


In [ ]:
# df.select("X", "CLASS").collect()

In [ ]:
# df.select("X").filter(df.CLASS == 2).show()

In [14]:
x0 = df.select("X").filter(df.CLASS == 1) # => Please create a DataFrame containing only measurements of class 0 from the x axis
y0 = df.select("Y").filter(df.CLASS == 1) # => Please create a DataFrame containing only measurements of class 0 from the y axis
z0 = df.select("Z").filter(df.CLASS == 1) # => Please create a DataFrame containing only measurements of class 0 from the z axis
x1 = df.select("X").filter(df.CLASS == 2) # => Please create a DataFrame containing only measurements of class 1 from the x axis
y1 = df.select("Y").filter(df.CLASS == 2) # => Please create a DataFrame containing only measurements of class 1 from the y axis
z1 = df.select("Z").filter(df.CLASS == 2) # => Please create a DataFrame containing only measurements of class 1 from the z axis

In [15]:
# x0 = df.select("X").filter(df.CLASS == 1)

In [16]:
type(x0)

pyspark.sql.dataframe.DataFrame

Since we’ve created this cool DFT function before, we can just call it for each of the 6 DataFrames now. And since the result of this function call is a DataFrame again we can use the pyspark best practice in simply calling methods on it sequentially. So what we are doing is the following:

- Calling DFT for each class and accelerometer sensor axis.
- Joining them together on the ID column. 
- Re-adding a column containing the class index.
- Stacking both Dataframes for each classes together



In [17]:
from pyspark.sql.functions import lit

df_class_0 = dft_systemml(x0,'x') \
    .join(dft_systemml(y0,'y'), on=['id'], how='inner') \
    .join(dft_systemml(z0,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(1))
    
df_class_1 = dft_systemml(x1,'x') \
    .join(dft_systemml(y1,'y'), on=['id'], how='inner') \
    .join(dft_systemml(z1,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(2))

df_dft = df_class_0.union(df_class_1)

df_dft.show()

SystemML Statistics:
Total execution time:		0.233 sec.
Number of executed Spark inst:	0.


SystemML Statistics:
Total execution time:		0.049 sec.
Number of executed Spark inst:	0.


SystemML Statistics:
Total execution time:		0.052 sec.
Number of executed Spark inst:	0.


[Stage 17:>                                                         (0 + 1) / 1]
                                                                                
[Stage 19:>                                                         (0 + 1) / 1]
SystemML Statistics:
Total execution time:		7.273 sec.
Number of executed Spark inst:	6.

                                                                                

[Stage 25:>                                                         (0 + 1) / 1]
[Stage 27:>                                                         (0 + 1) / 1]
SystemML Statistics:
Total execution time:		6.502 sec.
Number of executed Spark inst:	6.

                                                            

Please create a VectorAssembler which consumes the newly created DFT columns and produces a column “features”


In [18]:
from pyspark.ml.feature import VectorAssembler

In [19]:
vectorAssembler = VectorAssembler(
    inputCols=["xa", "xb", "ya", "yb", "za", "zb"],
    outputCol="features")

Please insatiate a classifier from the SparkML package and assign it to the classifier variable. Make sure to set the “class” column as target.


In [24]:
from pyspark.ml.classification import RandomForestClassifier

In [25]:
# classifier = GBTClassifier(labelCol="class", featuresCol="features", maxIter=10)
classifier = RandomForestClassifier(labelCol="class", featuresCol="features", numTrees=10)

Let’s train and evaluate…


In [26]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [27]:
model = pipeline.fit(df_dft)

In [28]:
prediction = model.transform(df_dft)

In [29]:
prediction.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
| id|                  xa|                  xb|                  ya|                  yb|                  za|                  zb|class|            features|       rawPrediction|         probability|prediction|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
| 26| 0.03912775214058598|-0.09049016952668232| 0.03912775214058598|-0.09049016952668232| 0.03912775214058598|-0.09049016952668232|    1|[0.03912775214058...|[0.0,8.5096832535...|[0.0,0.8509683253...|       1.0|
| 29|0.006260524476137005|-0.05765058448048809|0.006260524476137005|-0.05765058448048809|0.006260524476137005|-0.05765058448048809|    1|[0.006260524476

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("class")
    
binEval.evaluate(prediction) 

0.9981761070017225

If you are happy with the result (I’m happy with > 0.8) please submit your solution to the grader by executing the following cells, please don’t forget to obtain an assignment submission token (secret) from the Courera’s graders web page and paste it to the “secret” variable below, including your email address you’ve used for Coursera. 


In [31]:
!rm -Rf a2_m4.json

In [32]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m4.json')

In [33]:
!rm -f rklib.py
!wget wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2020-02-09 17:52:08--  http://wget/
Resolving wget (wget)... failed: nodename nor servname provided, or not known.
wget: unable to resolve host address ‘wget’
--2020-02-09 17:52:08--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0s      

2020-02-09 17:52:08 (10.3 MB/s) - ‘rklib.py’ saved [2540/2540]

FINISHED --2020-02-09 17:52:08--
Total wall clock time: 0.7s
Downloaded: 1 files, 2.5K in 0s (10.3 MB/s)


In [34]:
from rklib import zipit
zipit('a2_m4.json.zip','a2_m4.json')

In [35]:
!base64 a2_m4.json.zip > a2_m4.json.zip.base64

In [36]:
from rklib import submit
key = "-fBiYHYDEeiR4QqiFhAvkA"
part = "IjtJk"
email = ###YOUR_CODE_GOES_HERE###
submission_token = ###YOUR_CODE_GOES_HERE### # (have a look here if you need more information on how to obtain the token https://youtu.be/GcDo0Rwe06U?t=276)

with open('a2_m4.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, submission_token, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"B8wXV","id":"f_F-qCtuEei_fRLwaVDk3g~B8wXV~AgXcf0tlEeqXYhIP6SrxAw","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
